# Computation of top- and bottom-of-atmosphere reflectance

In this example, we demonstrate how to compute various reflectance quantities. We reference interested readers to Nicodemus et al. (1977) for a rigorous reflectance nomenclature.

> Nicodemus, Fred Edwin, J. C. Richmond, J. J. Hsia, I. W. Ginsberg, and T. Limperis. 1977. Geometrical Considerations and Nomenclature for Reflectance. Vol. 160. US Department of Commerce, National Bureau of Standards.

In practice, we will see how to set up simulations to compute radiance and incident flux values. We start with a few imports and setup operations:

In [ ]:
# First, import Eradiate
import eradiate

# Add supporting packages:
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np

# A few specific convenience imports
from eradiate.experiments import AtmosphereExperiment
from eradiate.tutorials import plot_polarfilm

# Select an operational mode
eradiate.set_mode("mono")

# Adjust plotting style (optional)
eradiate.plot.set_style()

In the following, we will perform many similar simulations that share common settings. Let's define them here and assign them to variables for convenience:

In [ ]:
# Result storage data structure
results = {}

# Plotting details — not critical
azimuth_convention = "north_left"  # the azimuth convention used for input and plots
spp = int(1e4)  # default sample count (increase e.g. to 1e5 to reduce noise)
vmin, vmax = 0.20, 0.50  # default bounds for color maps to make them comparable
levels = np.arange(
    vmin, vmax + 0.01, 0.02
)  # default levels for contour plots to make them comparable
theta_max = 80.0  # maximum zenith value for plots

# Common scene setup parameters
surface = {"type": "rpv"}
atmosphere = {
    "type": "heterogeneous",
    "molecular_atmosphere": {"type": "molecular"},
}
illumination = {
    "type": "directional",
    "zenith": 38.7,
    "azimuth": 0.0,
    "azimuth_convention": azimuth_convention,
}

## Surface BRF

We start by computing the bidirectional reflectance factor (BRF) of the surface alone. This quantity is sometimes referred to as *top-of-canopy BRF*. This quantity is output automatically by Eradiate when a distant sensor is used without modifying its `ray_offset` parameter:

In [ ]:
exp = AtmosphereExperiment(
    geometry="plane_parallel",
    surface=surface,
    atmosphere=None,
    illumination=illumination,
    measures={
        "type": "hdistant"
    },  # use an hdistant sensor to sample the entire hemisphere continuously
)
result = eradiate.run(exp, spp=spp)
results["toc_brf"] = result["brf"].squeeze(drop=True)
plt.show(
    plot_polarfilm(
        results["toc_brf"],
        azimuth_convention=azimuth_convention,
        vmin=vmin,
        vmax=vmax,
        levels=levels,
        theta_max=theta_max,
    )
)

The chosen surface reflection model is an RPV BRDF with a hotspot in the retroreflective direction.

Internally, Eradiate applies the definition of the BRF as given by Nicodemus et al. (1977):
$$
\text{BRF}
=
\text{BRDF} \times \pi
=
\pi \frac{L_\mathrm{r}}{E_\mathrm{i}}
$$
where $L_\mathrm{r}$ is the recorded reflected radiance (in W/m²/sr), and $E_\mathrm{i}$ is the incident flux as the surface (in W/m²). Variable dependencies are omitted for conciseness.

$L_\mathrm{r}$ is stored in the result dataset as the `radiance` variable, and $E_\mathrm{i}$ is stored as the `irradiance` variable. It is important to note that `irradiance` stores the horizontal solar irradiance at the top of the atmosphere. This means that it contains the cosine foreshortening factor (which decreases the flux when the illumination zenith angle increases). It also means that this variable is irrelevant if the point where the irradiance is needed is located somewhere in the atmosphere.

## Top-of-atmosphere BRF

A very commonly used quantity in Earth observation is the top-of-atmosphere BRF. To get it, we simply add an atmosphere to the previous setup. Eradiate automatically adjusts the simulation to make it consistent and return relevant results:

In [ ]:
exp = AtmosphereExperiment(
    geometry="plane_parallel",
    surface=surface,
    atmosphere=atmosphere,
    illumination=illumination,
    measures={"type": "hdistant"},
)
results["toa_brf_pp"] = eradiate.run(exp, spp=spp)["brf"].squeeze(drop=True)
plt.show(
    plot_polarfilm(
        results["toa_brf_pp"],
        azimuth_convention=azimuth_convention,
        vmin=vmin,
        vmax=vmax,
        levels=levels,
        theta_max=theta_max,
    )
)

We can see that the structure that was visible in the reflective pattern are faded, and that there is now Monte Carlo noise. This is due to atmospheric scattering. We are running these simulations at the default wavelength (550 nm), at which there is some Rayleigh scattering.

We can now modify this setup to compute the TOA BRF, accounting for planetary curvature. The most notable difference happens at grazing viewing angles, where we can see the curvature of the iso-lines change compared to the plane-parallel case.

In [ ]:
# TOA BRF (spherical-shell)

exp = AtmosphereExperiment(
    geometry="spherical_shell",
    surface=surface,
    atmosphere=atmosphere,
    illumination=illumination,
    measures={"type": "hdistant"},
)
results["toa_brf_ss"] = eradiate.run(exp, spp=spp)["brf"].squeeze(drop=True)
plt.show(
    plot_polarfilm(
        results["toa_brf_ss"],
        azimuth_convention=azimuth_convention,
        vmin=vmin,
        vmax=vmax,
        levels=levels,
        theta_max=theta_max,
    )
)

## Bottom-of-atmosphere hemispherical-directional reflectance factor

A hypothetical measurement of the BRF would require a directional illumination and sensor. This, in practice, can only be achieved (approximately) in a laboratory, illuminating a target with a collimated beam and recording the reflected radiance with a radiometer with a very narrow field of view.

When performing field measurements, an instrument is pointed at a target illuminated by the sky. Consequently, the only measurement that is available (again, approximately) factors in the illumination by the sky, which is generally far from directional. What we can compute in such conditions is the hemispherical-directional reflectance factor (HDRF).

Nicodemus et al. (1977) defines it as the integral of the BRDF on the incoming direction space. In practice, it can also be obtained by dividing a reflected radiance record by the total incident flux (this operation is identical to the one made to compute the BRF from our simulation results):

$$
\text{HDRF}
=
\pi \frac{L_\mathrm{r}}{E_\mathrm{i}}
$$

Eradiate can be used to compute the HDRF. Computing the reflected radiance $L_\mathrm{r}$ is straightforward; however, the scene setup prevents us from sampling easily the incident flux term $E_\mathrm{i}$. The reason is that to sample an emitter, it has to be visible in the field of view of the sensor — in other words, one needs to be able to "hit" it when casting a ray from the sensor; however, the illumination is perfectly directional, which means that it has an angular aperture exactly equal to 0, and that it is in practice never visible to a sensor.

We can however sample the emitter indirectly thanks to next event estimation (a.k.a. local estimate), which will sample the directional emitter at every scattering event. For that purpose, we add to the scene a flat, horizontal panel (*e.g.* a disk) with a diffuse reflectance equal to 1. All the incident radiation at the sensor is reflected without energy loss: if we sample it over the entire hemisphere, we indirectly sample the total incident flux at the surface (both the direct and diffuse components). The relationship between the incident flux and the reflected radiance is:

$$
E_\mathrm{i}
=
E_\mathrm{r}
=
\int_{2\pi} L_\mathrm{r,panel} (\omega_\mathrm{o}) \cos \theta_\mathrm{o} \mathrm{d} \omega_\mathrm{o}
=
\pi L_\mathrm{r,panel}
$$

where we take advantage of the fact that our panel has a diffuse reflectance, *i.e.* that $L_\mathrm{r,panel}$ has no angular dependency.

One more detail: if the panel is large, it will significantly perturb the diffuse radiation field, effectively biasing the incident radiative flux measurement. To reduce the perturbation to a negligible amount, we set the size of the panel to a very small value (*e.g.* 10 cm).

In [ ]:
# BOA HDRF (lambertian surface)
# This verifies that the methodology is correct

origin = np.array([0.0, 0.0, 0.0])
panel = origin + np.array(
    [100.0, 100.0, 0.05]
)  # the panel is 100 m away from the main measurement target

kdict = {
    "01_shape_reference_panel": {
        "type": "disk",
        "bsdf": {"type": "diffuse", "reflectance": {"type": "uniform", "value": 1.0}},
        "to_world": mi.ScalarTransform4f.translate(panel)
        @ mi.ScalarTransform4f.scale(0.01),  # the panel has radius ~1cm
    }
}

exp = AtmosphereExperiment(
    geometry="plane_parallel",
    surface={
        "type": "lambertian",
        "reflectance": 0.5,
    },  # our surface is Lambertian, with relfectance = 0.5
    atmosphere=atmosphere,
    illumination=illumination,
    measures=[
        {
            "type": "hdistant",
            "id": "origin",
            "target": origin,
            "ray_offset": 0.1,  # rays are not cast from an infinite distance:
            # they are cast from a close distance to the sensor target (10 cm)
        },
        {
            "type": "hdistant",
            "id": "panel",
            "target": panel,
            "ray_offset": 0.1,
        },
    ],
    kdict=kdict,
)

result = eradiate.run(exp, spp=spp)

hdrf_lambertian = (
    result["origin"]["radiance"].squeeze(drop=True)
    / result["panel"]["radiance"].squeeze(drop=True).mean()
    # Note 1: We average all L_{r,panel} values to reduce the noise
    # Note 2: The two π cancel out
)
plt.show(
    plot_polarfilm(
        hdrf_lambertian,
        azimuth_convention=azimuth_convention,
        show_contour=False,
        theta_max=theta_max,
    )
)

In [ ]:
# BOA HDRF (plane-parallel)
# We get back to the RPV setup

origin = np.array([0.0, 0.0, 0.0])
panel = origin + np.array([100.0, 100.0, 0.05])

kdict = {
    "01_shape_reference_panel": {
        "type": "disk",
        "bsdf": {"type": "diffuse", "reflectance": {"type": "uniform", "value": 1.0}},
        "to_world": mi.ScalarTransform4f.translate(panel)
        @ mi.ScalarTransform4f.scale(0.01),
    }
}

exp = AtmosphereExperiment(
    geometry="plane_parallel",
    surface=surface,
    atmosphere=atmosphere,
    illumination=illumination,
    measures=[
        {
            "type": "hdistant",
            "id": "origin",
            "target": origin,
            "ray_offset": 0.1,
        },
        {
            "type": "hdistant",
            "id": "panel",
            "target": panel,
            "ray_offset": 0.1,
        },
    ],
    kdict=kdict,
)

result = eradiate.run(exp, spp=spp)

results["boa_hdrf_pp"] = (
    result["origin"]["radiance"].squeeze(drop=True)
    / result["panel"]["radiance"].squeeze(drop=True).mean()
)
plt.show(
    plot_polarfilm(
        results["boa_hdrf_pp"],
        azimuth_convention=azimuth_convention,
        vmin=vmin,
        vmax=vmax,
        levels=levels,
        theta_max=theta_max,
    )
)

In [ ]:
# BOA HDRF (spherical-shell)

origin = np.array([0.0, 0.0, eradiate.constants.EARTH_RADIUS.m_as("m")])
panel = origin + np.array([100.0, 100.0, 0.05])

kdict = {
    "01_shape_reference_panel": {
        "type": "disk",
        "bsdf": {
            "type": "diffuse",
            "reflectance": {"type": "uniform", "value": 1.0},
        },
        "to_world": mi.ScalarTransform4f.translate(panel)
        @ mi.ScalarTransform4f.scale(0.01),
    }
}

exp = AtmosphereExperiment(
    geometry="spherical_shell",
    surface={"type": "rpv"},
    atmosphere=atmosphere,
    illumination=illumination,
    measures=[
        {
            "type": "hdistant",
            "id": "origin",
            "target": origin,
            "ray_offset": 0.1,
        },
        {
            "type": "hdistant",
            "id": "panel",
            "target": panel,
            "ray_offset": 0.1,
        },
    ],
    kdict=kdict,
)

result = eradiate.run(exp, spp=spp)

results["boa_hdrf_ss"] = (
    result["origin"]["radiance"].squeeze(drop=True)
    / result["panel"]["radiance"].squeeze(drop=True).mean()
)
plt.show(
    plot_polarfilm(
        results["boa_hdrf_ss"],
        azimuth_convention=azimuth_convention,
        vmin=vmin,
        vmax=vmax,
        levels=levels,
        theta_max=theta_max,
    )
)

## Viewing all of them

We can now plot all these reflectance quantities in the principal plane on a single line plot.

In [ ]:
np.unique(results["boa_hdrf_pp"].vza)

In [ ]:
PARAMS = {
    "toc_brf": {"label": "TOC BRF"},
    "toa_brf_pp": {"label": "TOA BRF (plane-parallel)"},
    "toa_brf_ss": {"label": "TOA BRF (spherical-shell)"},
    "boa_hdrf_pp": {"label": "BOA HDRF (plane-parallel)"},
    "boa_hdrf_ss": {"label": "BOA HDRF (spherical-shell)"},
}


for k, da in results.items():
    pplane = da.swap_dims({"x_index": "x"}).interp(x=0.5)
    negative_plane = (pplane.vaa > 180.0).values
    pplane.vza.values[negative_plane] = -pplane.vza.values[negative_plane]
    pplane = pplane.where(np.abs(pplane.vza) < theta_max)

    params = PARAMS[k]
    pplane.plot(marker=".", ls=":", x="vza", **params)

plt.title("")
plt.ylabel("")
plt.xlabel("Viewing zenith angle [°]")
plt.legend()

We can also visualize all polar plots together and compare patterns:

In [ ]:
for k, da in results.items():
    fig = plot_polarfilm(
        da,
        levels=levels,
        vmin=vmin,
        vmax=vmax,
        theta_max=theta_max,
        azimuth_convention=azimuth_convention,
    )
    fig.axes[1].set_title(PARAMS[k]["label"])
    plt.show(fig)

What to do next:

* If your machine allows it, try increasing the sample count to reduce the noise.
* The `hdistant` measure is not the best for the line plots, because it does not allow to place the viewing angles to the values we want: an `mdistant` measure would be better. Try setting up an `mdistant` measure that will capture the hotspot.

In [ ]:
# Paper figure

if False:
    PARAMS = {
        "toc_brf": {"label": "TOC BRF"},
        "toa_brf_pp": {"label": "TOA BRF"},
        "boa_hdrf_pp": {"label": "BOA HDRF"},
    }

    fig, ax = plt.subplots(1, 1, figsize=(6, 3))

    for k, da in results.items():
        try:
            params = PARAMS[k]
        except KeyError:
            continue

        pplane = da.swap_dims({"x_index": "x"}).interp(x=0.5)
        negative_plane = (pplane.vaa > 180.0).values
        pplane.vza.values[negative_plane] = -pplane.vza.values[negative_plane]
        pplane = pplane.where(np.abs(pplane.vza) < theta_max)
        pplane.plot(marker=".", ls=":", x="vza", **params)

    plt.title("")
    plt.ylabel("")
    plt.xlabel("Viewing zenith angle [°]")
    plt.legend()

    plt.savefig("reflectance.pdf", bbox_inches="tight")

    plt.show()
    plt.close()